# 12. Improving Simple CNN

In [1]:
import torch
import torch.nn as nn

import torchvision.transforms as transforms
import torchvision.datasets as dsets
import torch.utils.data as Data
import torch.nn.init as init

from torch.autograd import Variable
import numpy as np
import os

## 12.1 Preparing Custom Data

In [2]:
img_dir = "./data/jamo"
img_data = dsets.ImageFolder(img_dir, transforms.Compose([
            transforms.Grayscale(),
#           Data Augmentation
#           transforms.RandomRotation(15)
#           transforms.CenterCrop(28),
#           transforms.Lambda(lambda x: x.rotate(15)),
    
#           Data Nomalization
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.5,), std=(0.5,))
            ]))

#https://pytorch.org/docs/stable/torchvision/transforms.html

print(img_data.classes)
print(img_data.class_to_idx)

['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ', 'ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
{'ㄱ': 0, 'ㄲ': 1, 'ㄴ': 2, 'ㄷ': 3, 'ㄸ': 4, 'ㄹ': 5, 'ㅁ': 6, 'ㅂ': 7, 'ㅃ': 8, 'ㅅ': 9, 'ㅆ': 10, 'ㅇ': 11, 'ㅈ': 12, 'ㅉ': 13, 'ㅊ': 14, 'ㅋ': 15, 'ㅌ': 16, 'ㅍ': 17, 'ㅎ': 18, 'ㅏ': 19, 'ㅐ': 20, 'ㅑ': 21, 'ㅒ': 22, 'ㅓ': 23, 'ㅔ': 24, 'ㅕ': 25, 'ㅖ': 26, 'ㅗ': 27, 'ㅘ': 28, 'ㅙ': 29, 'ㅛ': 30, 'ㅜ': 31, 'ㅝ': 32, 'ㅞ': 33, 'ㅟ': 34, 'ㅠ': 35, 'ㅡ': 36, 'ㅢ': 37, 'ㅣ': 38}


In [3]:
batch_size = 100
font_num = 720

In [4]:
def train_test_split(data, train_ratio, stratify, stratify_num, batch_size) :
    
    length = len(data)
    
    if stratify :
        label_num = int(len(data)/stratify_num)
        cut = int(stratify_num*train_ratio)
        train_indices = np.random.permutation(np.arange(stratify_num))[:cut]
        test_indices = np.random.permutation(np.arange(stratify_num))[cut:]
        
        for i in range(1, label_num) :
            train_indices = np.concatenate((train_indices, np.random.permutation(np.arange(stratify_num))[:cut] + stratify_num*i))
            test_indices = np.concatenate((test_indices, np.random.permutation(np.arange(stratify_num))[cut:] + stratify_num*i))
        
    else :
        cut = int(len(data)*train_ratio)
        train_indices = np.random.permutation(np.arange(length))[:cut]
        test_indices = np.random.permutation(np.arange(length))[cut:]
        
    np.random.shuffle(test_indices)
    np.random.shuffle(train_indices)
    
    train_loader = Data.DataLoader(data, batch_size=batch_size, shuffle=False, sampler = train_indices, num_workers=1, drop_last = True)
    test_loader = Data.DataLoader(data, batch_size=batch_size, shuffle=False, sampler = test_indices, num_workers=1, drop_last = True)

    return train_loader, test_loader, len(train_indices), len(test_indices)

In [5]:
train_loader, test_loader, train_num, test_num = train_test_split(img_data, 0.8, True, font_num, batch_size)

## 12.2 Define Model

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer = nn.Sequential(
            nn.Conv2d(1,16,5),
            nn.ReLU(),
            #Dropout
            nn.Dropout2d(0.5),
            nn.Conv2d(16,32,5),
            #Batch Nomalization
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,64,5),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        
        self.fc_layer = nn.Sequential(
            nn.Linear(64*5*5,500),
            nn.ReLU(),
            nn.Linear(500,39)
        )
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
#                 Weight Initialization
#                 init.xavier_normal(m.weight.data)
                init.kaiming_normal(m.weight.data)
                m.bias.data.fill_(0)
        
            elif isinstance(m, nn.Linear):
                init.kaiming_normal(m.weight.data)
                m.bias.data.fill_(0)                
        
    def forward(self,x):
        out = self.layer(x)
        out = out.view(batch_size,-1)
        out = self.fc_layer(out)

        return out

model = CNN().cuda()

In [7]:
loss = nn.CrossEntropyLoss() # Loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Momentum & Weight Regularization(L2)
# optim.SGD(model.parameters(), lr=1e-2, momentum=0.9, weight_decay=1e-5)

In [8]:
num_epochs = 50

In [9]:
def test_model() :
    
    model.eval()
    
    correct = 0
    total = 0

    for images, labels in test_loader:

        images = Variable(images).cuda()
        outputs = model(images)

        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels.cuda()).sum()

    print('Accuracy of test images: %f %%' % (100 * correct / total))

In [10]:
# Learning Rate Scheduler
import torch.optim.lr_scheduler as lr_scheduler
#scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma= 0.99)
#scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[10,30,80], gamma= 0.1)
scheduler = lr_scheduler.ExponentialLR(optimizer, gamma= 0.99)
#scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min')

for epoch in range(num_epochs):
    scheduler.step()
    for i, (batch_images, batch_labels) in enumerate(train_loader):
        
        X = Variable(batch_images).cuda()
        Y = Variable(batch_labels).cuda()

        pre = model(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [%d/%d], lter [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, train_num//batch_size, cost.data[0]))
            test_model()
            model.train()

Epoch [1/50], lter [100/224], Loss: 0.7463
Accuracy of test images: 72.892857 %
Epoch [1/50], lter [200/224], Loss: 0.4524
Accuracy of test images: 81.821429 %
Epoch [2/50], lter [100/224], Loss: 0.2912
Accuracy of test images: 86.982143 %
Epoch [2/50], lter [200/224], Loss: 0.2548
Accuracy of test images: 87.035714 %
Epoch [3/50], lter [100/224], Loss: 0.2339
Accuracy of test images: 90.446429 %
Epoch [3/50], lter [200/224], Loss: 0.1254
Accuracy of test images: 92.982143 %
Epoch [4/50], lter [100/224], Loss: 0.0695
Accuracy of test images: 92.178571 %
Epoch [4/50], lter [200/224], Loss: 0.2028
Accuracy of test images: 95.107143 %
Epoch [5/50], lter [100/224], Loss: 0.1206
Accuracy of test images: 96.392857 %
Epoch [5/50], lter [200/224], Loss: 0.0541
Accuracy of test images: 97.142857 %
Epoch [6/50], lter [100/224], Loss: 0.0734
Accuracy of test images: 97.732143 %
Epoch [6/50], lter [200/224], Loss: 0.1048
Accuracy of test images: 97.339286 %
Epoch [7/50], lter [100/224], Loss: 0.06